In [1]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
from tqdm import tqdm
from multiprocess import Process, Manager
from typing import List, Dict
import json
from joblib import delayed, Parallel
import itertools
from itertools import chain
import pickle as pkl

In [2]:
df_results_from_query = pd.DataFrame(pkl.load(open("results_with_names_doi_relationship.pkl", "rb")))
df_results_from_query.iloc[1518] #["title"]

project_id                                                      96/05497-3
orcids                   [https://orcid.org/0000-0001-5203-4366, https:...
title                                        Pesquisa em visão cibernética
summary                  O presente projeto visa suportar as atividades...
keywords                 [Inteligência artificial, Visão computacional,...
related_dois                                                            []
pubs_names               [Optimized approach to multiscale skeleton gen...
name_doi_relationship    {'Optimized approach to multiscale skeleton ge...
Name: 1518, dtype: object

In [3]:
papers_names_with_dois = [eval(str(x)) for x in df_results_from_query["name_doi_relationship"]]

In [4]:
not_recovered_papers = []
recovered_papers = []
for x, (_, row) in zip(papers_names_with_dois, df_results_from_query.iterrows()): 
    for k,v in x.items():
        if isinstance(v, list):
            not_recovered_papers.append((k, row))
        elif isinstance(v, str):
            if v == 'https://www.doi.org/' or v == 'http://www.doi.org/':
                not_recovered_papers.append((k, row))
            else:
                recovered_papers.append((k, v))

In [5]:
len(recovered_papers) / (len(not_recovered_papers) + len(recovered_papers))

0.42898081235983054

In [6]:
recovered_papers

[('State Disaggregation for Dynamic Scaling of Network Functions',
  'https://www.doi.org/10.1109/TNET.2023.3282562'),
 ('Analysis of functional connectivity using machine learning and deep learning in different data modalities from individuals with schizophrenia',
  'https://www.doi.org/10.1088/1741-2552/acf734'),
 ('Neural architecture search with interpretable meta-features and fast predictors',
  'https://www.doi.org/10.1016/j.ins.2023.119642'),
 ('Slicing who slices: Anonymization quality evaluation on deployment, privacy, and utility in mix-zones',
  'https://www.doi.org/10.1016/j.comnet.2023.110007'),
 ('Towards augmented and mixed reality on future mobile networks',
  'https://www.doi.org/10.1007/s11042-023-15301-4'),
 ('On strategies to help reduce contamination on public transit: a multilayer network approach',
  'https://www.doi.org/10.1007/s41109-023-00562-7'),
 ('A Novel Mechanism for Misbehavior Detection in Vehicular Networks',
  'https://www.doi.org/10.1109/ACCESS.2023.

In [7]:
recovered_papers_open_alex = []
for x in tqdm(recovered_papers, total=len(recovered_papers)):
    try:
        recovered_papers_open_alex.append(Works()[x[1]])
    except:
        pass

100%|█████████████████████████████| 3443/3443 [42:29<00:00,  1.35it/s]


In [8]:
len(recovered_papers_open_alex) / (len(not_recovered_papers) + len(recovered_papers))

0.41913780214303514

In [9]:
not_recovered_papers_recovering_by_title = []
for p in tqdm(not_recovered_papers, total=len(not_recovered_papers)):
    try:
        not_recovered_papers_recovering_by_title.append(
            (p[0], Works().search_filter(title=p[0]).get()[0])
        )
    except:
        pass

100%|███████████████████████████| 4583/4583 [1:06:24<00:00,  1.15it/s]


In [10]:
import pickle as pkl
pkl.dump(recovered_papers_open_alex, open("recovered_papers_open_alex.pkl", "wb"))
pkl.dump(not_recovered_papers_recovering_by_title, open("not_recovered_papers_recovering_by_title.pkl", "wb"))

In [11]:
(len(recovered_papers_open_alex)+len(not_recovered_papers_recovering_by_title)) / (len(not_recovered_papers) + len(recovered_papers))

0.9438076252180414

In [17]:
not_recovered_papers_recovering_by_title[0]

('Pruning and Early-Exit Co-Optimization for CNN Acceleration on FPGAs',
 {'id': 'https://openalex.org/W4379115861',
  'doi': 'https://doi.org/10.23919/date56975.2023.10137244',
  'title': 'Pruning and Early-Exit Co-Optimization for CNN Acceleration on FPGAs',
  'display_name': 'Pruning and Early-Exit Co-Optimization for CNN Acceleration on FPGAs',
  'relevance_score': 90.27194,
  'publication_year': 2023,
  'publication_date': '2023-04-01',
  'ids': {'openalex': 'https://openalex.org/W4379115861',
   'doi': 'https://doi.org/10.23919/date56975.2023.10137244'},
  'language': 'en',
  'primary_location': {'is_oa': False,
   'landing_page_url': 'https://doi.org/10.23919/date56975.2023.10137244',
   'pdf_url': None,
   'source': None,
   'license': None,
   'version': None,
   'is_accepted': False,
   'is_published': False},
  'type': 'article',
  'type_crossref': 'proceedings-article',
  'indexed_in': ['crossref'],
  'open_access': {'is_oa': False,
   'oa_status': 'closed',
   'oa_url': No

In [4]:
df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)
df = df[df["Beneficiário"].notna() | df["Pesquisador Responsável"].notna()].reset_index(drop=True)
df

/tmp/ipykernel_402378/3775867619.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)


,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Assuntos,Data de Início,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Inst. Ext. / Convênio),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados
0,23/04895-4,O número de Ramsey relativo a arestas e grafos...,Size-Ramsey numbers and Ramsey-finiteness of g...,Meysam Miralaei,Universidade de São Paulo (USP). Instituto de ...,São Paulo,NaN,NaN,NaN,Yoshiharu Kohayakawa,...,Combinatória:Teoria de Ramsey:Grafos:Métodos p...,2024-05-01,2026-04-30,NaN,NaN,NaN,NaN,Este projeto tem foco em problemas sobre o núm...,This project focuses on numerical problems inv...,NaN
1,23/17002-8,Verificação de identidade de Usuários em redes...,User Identity Verification in CBDC Networks Us...,João Pedro Alonso Almeida,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Jó Ueyama,...,Blockchain,2024-03-25,2024-07-24,NaN,Queensland University Of Technology,Austrália,Raja Jurdak,Uma Moeda Digital de Banco Central (CBDC) repr...,A Central Bank Digital Currency (CBDC) represe...,NaN
2,23/13688-2,An Autoencoder model for dealing with missing ...,Um modelo Autoencoder para lidar com dados aus...,Arthur Dantas Mangussi,Ministério da Defesa (Brasil). Instituto Tecno...,São José dos Campos,NaN,NaN,NaN,Ana Carolina Lorena,...,Aprendizado computacional:Pré-processamento de...,2024-03-15,2024-09-14,NaN,Universidade de Coimbra (UC),Portugal,Pedro Henriques Abreu,Vários problemas podem deteriorar a qualidade ...,Various issues can deteriorate data quality in...,NaN
3,23/10398-3,HUB FUNDOS: plataforma computacional para a pa...,HUB FUNDOS: computational platform for the sta...,Aloisio Mota Rodrigues Junior,NaN,NaN,NaN,RTM Infraestrutura em Tecnologia da Informação...,São Paulo,Aloisio Mota Rodrigues Junior,...,Processamento de linguagem natural:Transformaç...,2024-03-01,2024-11-30,NaN,NaN,NaN,NaN,"Nos últimos dez anos, a indústria de Fundos no...","Over the last ten years, the Funds industry in...",NaN
4,23/06461-1,O uso de videogames no monitoramento e na manu...,The use of video games in monitoring and maint...,Fabio Ota,NaN,NaN,NaN,Eliza Massako Akiyama Ota,São Paulo,Fabio Ota,...,Inteligência artificial:Jogos de computador:Lo...,2024-03-01,2025-02-28,NaN,NaN,NaN,NaN,O envelhecimento populacional é um fenômeno ob...,Population aging is a phenomenon observed worl...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7492,92/01292-7,Desenvolvimento e implementação de algoritmo p...,NaN,Edson Roberto Minatel,"Ministério da Agricultura, Pecuária e Abasteci...",São Carlos,NaN,NaN,NaN,Paulo Estevão Cruvinel,...,Transformada de Fourier:Algoritmos,1992-07-01,1993-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7493,92/01362-5,NaN,A scheduling stratagy for a distributed hard r...,Maurício Ferreira Magalhães,Universidade Estadual de Campinas (UNICAMP). F...,Campinas,NaN,NaN,NaN,Maurício Ferreira Magalhães,...,NaN,1992-06-23,1992-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7494,92/01118-7,Computação paralela em álgebra linear numérica,NaN,Edna Yoshiko Senzako,Universidade Estadual Paulista (UNESP). Campus...,São José do Rio Preto,NaN,NaN,NaN,Odelar Leite Linhares,...,Álgebra linear,1992-06-01,1993-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7495,92/00542-0,Autômatos finitos: teoria e prática,NaN,Imre Simon,Universidade de São Paulo (USP). Instituto de ...,São Paulo,NaN,NaN,NaN,Imre Simon,...,Autômatos finitos:Compressão de dados:Entropia...,1992-05-30,1992-07-13,NaN,Université Paris Diderot - Paris 7,França,Dominique Perrin,Pretendo prosseguir e aprofundar as minhas pes...,NaN,NaN


In [5]:
df = df.merge(df_results_from_query[["project_id", "orcids", "name_doi_relationship"]].rename(columns={"project_id": "N. Processo"}), on="N. Processo", how="left")

In [7]:
df.dropna(subset="name_doi_relationship")

,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Inst. Ext. / Convênio),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados,orcids,name_doi_relationship
144,23/00812-7,Aceleração de aplicações sensíveis a atrasos e...,Acceleration of latency-sensitive applications...,Ronaldo Alves Ferreira,Universidade Federal de Mato Grosso do Sul (UF...,Campo Grande,NaN,NaN,NaN,Ronaldo Alves Ferreira,...,2025-08-31,MCTI/MC,NaN,NaN,NaN,Aplicações em nuvem são complexas e normalment...,Cloud applications are complex and usually dec...,NaN,"[https://orcid.org/0000-0002-9144-7187, https:...",{'State Disaggregation for Dynamic Scaling of ...
163,22/14503-3,Serviços de transporte inteligentes para as re...,Intelligent transport services for the future ...,Bruno Yuji Lino Kimura,Universidade Federal de São Paulo (UNIFESP). C...,São José dos Campos,NaN,NaN,NaN,Bruno Yuji Lino Kimura,...,2024-07-31,NaN,King's College London,Inglaterra,Toktam Mahmoodi,"As futuras redes 6G são esperadas para 2030, q...","The future 6G networks are expected by 2030, w...",NaN,"[https://orcid.org/0000-0002-2174-2762, https:...",{'Towards augmented and mixed reality on futur...
165,23/00148-0,SocialNet: sensoriamento social para alavancar...,SocialNet: social sensing to leverage new tech...,Thiago Henrique Silva,Universidade Tecnológica Federal do Paraná (UT...,Curitiba,NaN,NaN,NaN,Thiago Henrique Silva,...,2025-07-31,MCTI/MC,NaN,NaN,NaN,"Novas fontes massivas de dados, como o Waze e ...","New massive sources of data, such as Waze and ...",23/12259-0; 23/12345-4,"[https://orcid.org/0000-0001-6994-8076, https:...",{'On strategies to help reduce contamination o...
167,23/08420-0,Um novo mecanismo para detecção de mau comport...,NaN,Rodolfo Ipolito Meneguette,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Rodolfo Ipolito Meneguette,...,2024-01-31,NaN,NaN,NaN,NaN,Os Sistemas de Transporte Inteligente (STI) fo...,Intelligent Transport Systems (ITS) have provi...,NaN,"[https://orcid.org/0000-0003-2982-4006, https:...",{'A Novel Mechanism for Misbehavior Detection ...
180,23/00721-1,Quantificação de incerteza em aprendizado fede...,Quantifying uncertainty in adversarial federat...,Heitor Soares Ramos Filho,Universidade Federal de Minas Gerais (UFMG). I...,Belo Horizonte,NaN,NaN,NaN,Heitor Soares Ramos Filho,...,2025-07-31,MCTI/MC,NaN,NaN,NaN,O projeto de pesquisa denominado Quantifying U...,The research project called Quantifying Uncert...,NaN,"[https://orcid.org/0000-0003-4523-6466, https:...",{'Slicing who slices: Anonymization quality ev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,96/07200-8,"Abordagens iterativas em problemas inversos, c...","Iterative approaches in inverse problems, with...",Pedro Paulo Balbi de Oliveira,Universidade do Vale do Paraíba (UNIVAP). Inst...,São José dos Campos,NaN,NaN,NaN,Pedro Paulo Balbi de Oliveira,...,2000-03-31,NaN,NaN,NaN,NaN,Serão abordadas três linhas de pesquisa conect...,Three lines of research will be tackled which ...,97/00944-4; 97/04825-0; 97/09663-8; 97/14303-0...,"[https://orcid.org/0000-0002-6022-0270, https:...",{'Inversão numérica de distribuições bidimensi...
7170,96/05497-3,Pesquisa em visão cibernética,NaN,Luciano da Fontoura Costa,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Luciano da Fontoura Costa,...,2001-02-28,NaN,NaN,NaN,NaN,O presente projeto visa suportar as atividades...,NaN,97/06871-9; 98/06877-0; 98/10700-8; 99/05855-5,"[https://orcid.org/0000-0001-5203-4366, https:...",{'Optimized approach to multiscale skeleton ge...
7194,96/04505-2,Aspectos estruturais e algorítmicos de objetos...,Structural and algorithmic aspects of combinat...,Yoshiharu Kohayakawa,Universidade

In [14]:
Authors()["https://orcid.org/0000-0002-5591-3750"]

{'id': 'https://openalex.org/A5023928945',
 'orcid': 'https://orcid.org/0000-0002-5591-3750',
 'display_name': 'Jó Ueyama',
 'display_name_alternatives': ['Jó Ueyama',
  'Jo Ueyama',
  'UeyamaJo',
  'J. Ueyama'],
 'works_count': 158,
 'cited_by_count': 2212,
 'summary_stats': {'2yr_mean_citedness': 3.111111111111111,
  'h_index': 23,
  'i10_index': 45},
 'ids': {'openalex': 'https://openalex.org/A5023928945',
  'orcid': 'https://orcid.org/0000-0002-5591-3750'},
 'affiliations': [{'institution': {'id': 'https://openalex.org/I17974374',
    'ror': 'https://ror.org/036rp1748',
    'display_name': 'Universidade de São Paulo',
    'country_code': 'BR',
    'type': 'education',
    'lineage': ['https://openalex.org/I17974374']},
   'years': [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015]},
  {'institution': {'id': 'https://openalex.org/I4210131883',
    'ror': 'https://ror.org/03kcw4w74',
    'display_name': 'Brazilian Society of Computational and Applied Mathematics',
    'coun